In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random
import re

base_url = "https://fbref.com"
main_url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
headers = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/88.0.4324.150 Safari/537.36")
}

# 1) Regex Patterns
# ---------------------------------------------------
# Passing Accuracy (two patterns)
passing_pattern_1 = re.compile(r"(\d+)\s+of\s+(\d+)(?:[^\d]+)(\d+)%")
passing_pattern_2 = re.compile(r"(\d+)%(?:[^\d]+)(\d+)\s+of\s+(\d+)")

# Shots on Target (two patterns)
shots_pattern_1 = re.compile(r"(\d+)\s+of\s+(\d+)(?:[^\d]+)(\d+)%?")
shots_pattern_2 = re.compile(r"(\d+)%(?:[^\d]+)(\d+)\s+of\s+(\d+)")

# Saves (two patterns)
saves_pattern_1 = re.compile(r"(\d+)\s+of\s+(\d+)(?:[^\d]+)(\d+)%")
saves_pattern_2 = re.compile(r"(\d+)%(?:[^\d]+)(\d+)\s+of\s+(\d+)")

def parse_passing(text):
    # "280 of 393 — 71%" => pattern_1
    # "71% — 280 of 393" => pattern_2
    m1 = passing_pattern_1.search(text)
    if m1:
        return (m1.group(1), m1.group(2))  # (completed, total)
    m2 = passing_pattern_2.search(text)
    if m2:
        return (m2.group(2), m2.group(3))
    return (None, None)

def parse_shots(text):
    # "8 of 18 — 44%" => pattern_1
    # "44% — 8 of 18" => pattern_2
    m1 = shots_pattern_1.search(text)
    if m1:
        return (m1.group(1), m1.group(2))  # (on_target, total)
    m2 = shots_pattern_2.search(text)
    if m2:
        return (m2.group(2), m2.group(3))
    return (None, None)

def parse_saves(text):
    # "4 of 7 — 57%" => pattern_1
    # "57% — 4 of 7" => pattern_2
    m1 = saves_pattern_1.search(text)
    if m1:
        return m1.group(3)  # just the percentage
    m2 = saves_pattern_2.search(text)
    if m2:
        return m2.group(1)
    return None

# 2) Main scraping code
# ---------------------------------------------------
response = requests.get(main_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
matches = soup.find_all("tr")

match_data = []

for match in matches:


    row_classes = match.get("class", [])
    if "spacer" in row_classes or "partial_table_header" in row_classes:
        continue

    home_team_el = match.find("td", {"data-stat": "home_team"})
    away_team_el = match.find("td", {"data-stat": "away_team"})
    score_el = match.find("td", {"data-stat": "score"})
    if not (home_team_el and away_team_el and score_el):
        continue

    home_team = home_team_el.text.strip()
    away_team = away_team_el.text.strip()
    score = score_el.text.strip()

    match_report_cell = match.find("td", {"data-stat": "match_report"})
    if not match_report_cell:
        continue
    link_tag = match_report_cell.find("a")
    if not link_tag:
        continue

    match_url = base_url + link_tag["href"]
    print(f"Scraping: {home_team} {score} {away_team}")
    print(match_url)

    res = requests.get(match_url, headers=headers)
    soup_match = BeautifulSoup(res.text, "html.parser")

    # 1) "team_stats_extra" (fouls, corners, etc.)
    stats_section = soup_match.find("div", {"id": "team_stats_extra"})
    if not stats_section:
        print(f"⚠ Stats section missing for {home_team} vs {away_team}")
        continue

    grids = [g for g in stats_section.find_all("div", recursive=False) if g.find("div")]
    if not grids:
        print(f"⚠ No grid blocks found for {home_team} vs {away_team}")
        continue

    home_stats = {}
    away_stats = {}

    # Extract from team_stats_extra
    for grid in grids:
        divs = grid.find_all("div")
        if len(divs) < 3:
            continue
        for i in range(3, len(divs), 3):
            if i + 2 < len(divs):
                home_val = divs[i].text.strip()
                stat_name = divs[i + 1].text.strip()
                away_val = divs[i + 2].text.strip()
                home_stats[stat_name] = home_val
                away_stats[stat_name] = away_val

    # 2) "team_stats" (possession, passing, shots on target, saves, cards)
    team_stats_section = soup_match.find("div", {"id": "team_stats"})
    if team_stats_section:
        table = team_stats_section.find("table")
        if table:
            body_or_table = table.find("tbody") or table
            rows = body_or_table.find_all("tr")

            i = 0
            while i < len(rows) - 1:
                label_th = rows[i].find("th", colspan="2")
                if label_th:
                    label = label_th.get_text(strip=True)
                    data_row = rows[i+1]
                    tds = data_row.find_all("td")
                    if len(tds) >= 2:
                        text_home = tds[0].get_text(strip=True)
                        text_away = tds[1].get_text(strip=True)

                        if label == "Possession":
                            home_stats["Possession"] = text_home.replace('%','')
                            away_stats["Possession"] = text_away.replace('%','')

                        elif label == "Passing Accuracy":
                            h_completed, h_total = parse_passing(text_home)
                            a_completed, a_total = parse_passing(text_away)
                            if h_completed and h_total:
                                home_stats["Passes Completed"] = h_completed
                                home_stats["Total Passes"] = h_total
                            if a_completed and a_total:
                                away_stats["Passes Completed"] = a_completed
                                away_stats["Total Passes"] = a_total

                        elif label == "Shots on Target":
                            h_on, h_tot = parse_shots(text_home)
                            a_on, a_tot = parse_shots(text_away)
                            if h_on and h_tot:
                                home_stats["Shots on Target"] = h_on
                                home_stats["Total Shots"] = h_tot
                            if a_on and a_tot:
                                away_stats["Shots on Target"] = a_on
                                away_stats["Total Shots"] = a_tot

                        elif label == "Saves":
                            sp_home = parse_saves(text_home)
                            sp_away = parse_saves(text_away)
                            if sp_home:
                                home_stats["Saves Percentage"] = sp_home
                            if sp_away:
                                away_stats["Saves Percentage"] = sp_away

                        elif label == "Cards":
                            home_yellow = len(tds[0].find_all("span", class_="yellow_card"))
                            away_yellow = len(tds[1].find_all("span", class_="yellow_card"))
                            home_red = len(tds[0].find_all("span", class_="red_card"))
                            away_red = len(tds[1].find_all("span", class_="red_card"))
                            home_stats["Yellow Cards"] = home_yellow
                            away_stats["Yellow Cards"] = away_yellow
                            home_stats["Red Cards"] = home_red
                            away_stats["Red Cards"] = away_red

                    i += 2
                else:
                    i += 1

    home_info = {
        "Team": home_team,
        "Opponent": away_team,
        "Match Result": score,
        "Home/Away": "Home"
    }
    away_info = {
        "Team": away_team,
        "Opponent": home_team,
        "Match Result": score,
        "Home/Away": "Away"
    }
    home_info.update(home_stats)
    away_info.update(away_stats)

    match_data.append(home_info)
    match_data.append(away_info)

    time.sleep(random.uniform(2, 6))

df = pd.DataFrame(match_data)
csv_filename = "/content/drive/MyDrive/3rd year 2nd Semester/Bets model/Premierbot/statspermatch.csv"
df.to_csv(csv_filename, index=False)

print("\nExtracted Match Data:")
print(df.head(10))
print(f"\n✅ Data saved to {csv_filename}")


Scraping: Manchester Utd 1–0 Fulham
https://fbref.com/en/matches/cc5b4244/Manchester-United-Fulham-August-16-2024-Premier-League
Scraping: Ipswich Town 0–2 Liverpool
https://fbref.com/en/matches/a1d0d529/Ipswich-Town-Liverpool-August-17-2024-Premier-League
Scraping: Newcastle Utd 1–0 Southampton
https://fbref.com/en/matches/34557647/Newcastle-United-Southampton-August-17-2024-Premier-League
Scraping: Nott'ham Forest 1–1 Bournemouth
https://fbref.com/en/matches/4efc72e4/Nottingham-Forest-Bournemouth-August-17-2024-Premier-League
Scraping: Everton 0–3 Brighton
https://fbref.com/en/matches/71618ace/Everton-Brighton-and-Hove-Albion-August-17-2024-Premier-League
Scraping: Arsenal 2–0 Wolves
https://fbref.com/en/matches/c0e3342a/Arsenal-Wolverhampton-Wanderers-August-17-2024-Premier-League
Scraping: West Ham 1–2 Aston Villa
https://fbref.com/en/matches/eac7c00b/West-Ham-United-Aston-Villa-August-17-2024-Premier-League
Scraping: Brentford 2–1 Crystal Palace
https://fbref.com/en/matches/b63822

In [3]:
structured = pd.read_csv('/content/drive/MyDrive/3rd year 2nd Semester/Bets model/Premierbot/statspermatch.csv')
#add a new column with matchday
#matchday entries should start with 1 and every row that is multiple of 20 should add one to the counter so
#from row 20 to row 39, entries should be 2 but then from 40 to 59 it should be 3 and so on
matchday = 1
structured1 = structured
structured1['Matchday'] = 1
for i in range(1, len(structured1)):
    if i % 20 == 0:
        matchday += 1

    structured1.at[i, 'Matchday'] = matchday
structured1

,Team,Opponent,Match Result,Home/Away,Fouls,Corners,Crosses,Touches,Tackles,Interceptions,...,Long Balls,Possession,Passes Completed,Total Passes,Shots on Target,Total Shots,Saves Percentage,Yellow Cards,Red Cards,Matchday
0,Manchester Utd,Fulham,1–0,Home,12,7,18,640,21,19,...,48,55,430,524,5,14,100.0,2,0,1
1,Fulham,Manchester Utd,1–0,Away,10,8,21,540,24,9,...,52,45,329,424,2,10,80.0,3,0,1
2,Ipswich Town,Liverpool,0–2,Home,9,2,8,497,20,9,...,57,38,290,381,2,7,60.0,3,0,1
3,Liverpool,Ipswich Town,0–2,Away,18,10,23,720,9,5,...,70,62,520,620,5,18,100.0,1,0,1
4,Newcastle Utd,Southampton,1–0,Home,15,3,12,317,16,4,...,45,23,128,207,1,3,100.0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,Chelsea,Aston Villa,2–1,Away,16,3,12,592,16,10,...,56,48,409,475,6,15,66.0,3,0,26
518,Newcastle Utd,Nott'ham Forest,4–3,Home,11,7,18,645,20,11,...,54,57,407,512,4,12,40.0,1,0,26
519,Nott'ham Forest,Newcastle Utd,4–3,Away,13,6,20,531,28,3,...,64,43,279,386,5,17,25.0,4,0,26
520,Manchester City,Liverpool,0–2,Home,3,7,22,804,5,3,...,36,66,627,704,5,16,50.0,0,0,27


In [ ]:
structured2 = structured1.copy()
# Create an empty row with NaN values
empty_row = pd.Series([None] * len(df.columns), index=df.columns)

# Insert the empty row every day matchday entry is not equal to the one before
#if the one before is a null value, continue
for i in range(1, len(structured2)):
    if structured2.at[i, 'Matchday'] != structured2.at[i-1, 'Matchday']:
        if pd.isnull(structured2.at[i-1, 'Matchday']):
            continue
        insert_index = i
        structured2.loc[insert_index] = None

        # Sort index to maintain order
        structured2 = structured2.sort_index().reset_index(drop=True)
structured2

,Team,Opponent,Match Result,Home/Away,Fouls,Corners,Crosses,Touches,Tackles,Interceptions,...,Long Balls,Possession,Passes Completed,Total Passes,Shots on Target,Total Shots,Saves Percentage,Yellow Cards,Red Cards,Matchday
0,Manchester Utd,Fulham,1–0,Home,12.0,7.0,18.0,640.0,21.0,19.0,...,48.0,55.0,430.0,524.0,5.0,14.0,100.0,2.0,0.0,1.0
1,Fulham,Manchester Utd,1–0,Away,10.0,8.0,21.0,540.0,24.0,9.0,...,52.0,45.0,329.0,424.0,2.0,10.0,80.0,3.0,0.0,1.0
2,Ipswich Town,Liverpool,0–2,Home,9.0,2.0,8.0,497.0,20.0,9.0,...,57.0,38.0,290.0,381.0,2.0,7.0,60.0,3.0,0.0,1.0
3,Liverpool,Ipswich Town,0–2,Away,18.0,10.0,23.0,720.0,9.0,5.0,...,70.0,62.0,520.0,620.0,5.0,18.0,100.0,1.0,0.0,1.0
4,Newcastle Utd,Southampton,1–0,Home,15.0,3.0,12.0,317.0,16.0,4.0,...,45.0,23.0,128.0,207.0,1.0,3.0,100.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,Wolves,Liverpool,2–1,Away,13.0,3.0,10.0,625.0,15.0,9.0,...,47.0,50.0,406.0,489.0,4.0,16.0,50.0,2.0,0.0,25.0
498,Tottenham,Manchester Utd,1–0,Home,13.0,10.0,31.0,712.0,13.0,5.0,...,36.0,55.0,482.0,566.0,7.0,22.0,100.0,1.0,0.0,25.0
499,Manchester Utd,Tottenham,1–0,Away,9.0,5.0,24.0,605.0,32.0,11.0,...,63.0,45.0,367.0,461.0,6.0,16.0,85.0,2.0,0.0,25.0
500,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
structured2.loc[2, "Match Result"]

'0–2'

In [6]:
structured1.to_excel('/content/drive/MyDrive/3rd year 2nd Semester/Bets model/Premierbot/statspermatchfinal.xlsx', index=False)